# 4. 파이토치로 구현한 RNN

## 4-1. 영화리뷰 RNN(BasicGRU)으로 감정분석하기

In [22]:
!pip install torchtext==0.6.0

  Using cached torchtext-0.6.0-py3-none-any.whl.metadata (6.3 kB)
Using cached torchtext-0.6.0-py3-none-any.whl (64 kB)


In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import datasets,data

BATCH_SIZE = 64
lr = 0.001
EPOCHS = 6
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## 데이터를 로드할때 어떻게 전처리를 진행할지를 정의하는 객체 Field
### sequential : 시퀀스 데이터 여부 (텍스트 데이터의 경우 True)
### batch_first : 미니배치 차원을 맨 앞으로 하여 데이터를 불러올지 여부 
TEXT = data.Field(sequential=True,batch_first=True,lower=True)
LABEL = data.Field(sequential=False,batch_first=True)

trainset,testset = datasets.IMDB.splits(TEXT,LABEL)

## 단어 사전을 만듬
### min_freq = 5 : 최소 5번 이상 등장한 단어만을 사전에 담겠다는 의미
### 5번 미만으로 출현하는 단어는 unk 토큰으로 대체
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

trainset,valset = trainset.split(split_ratio=0.8)

## 반복자 생성 => 반복할때마다 배치를 생성시켜주는 반복자
train_iter,val_iter,test_iter = data.BucketIterator.splits(
    (trainset,valset,testset), batch_size=BATCH_SIZE, shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2

class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        ## RNN은 Vanishing gradient 문제가 발생할 수 있기 때문에 GRU를 사용
        ## GRU는 시계열 데이터 속 벡터사이의 정보 전달량을 조절함으로써 기울기를 적정하게 유지하고 문장 앞부분의 정보가 끝까지 도달 할 수 있도록 도와줌
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        ## RNN은 입력데이터 X외에도 초기 은닉 상태를 입력해야함
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.gru(x, h_0)
        ## 마지막 time-step의 hidden state를 추출 => 마지막 선형 layer 들어가기전 히든 스테이트
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    
    def _init_state(self, batch_size=1):
        ## parameters 함수는 그 신경망 모듈의 가중치 정보들을 반복자 형태로 반환
        ## data 함수는 그 가중치(텐서) 정보들을 실제 값으로 반환
        ## 즉 아래 코드는 gru 모듈의 첫번째 가중치 텐서(모델의 가중치 텐서와 같은 데이터 타입)를 추출함
        weight = next(self.parameters()).data
        ## new 함수는 텐서를 생성하는 함수
        ## 모델의 가중치와 같은 모양(n_layers,batch_size,hidden_dim)의 텐서로 변환
        ## 텐서의 모든 값을 0으로 초기화 => zero_() 함수
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

def train(model,optimizer,train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        ## 모든 y 값에서 1을 빼는 것 ( 1,2 를 0,1로 변환)
        y.data.sub_(1)
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

def evaluate(model,eval_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in eval_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += logit.max(1)[1].eq(y).sum().item()
    size = len(eval_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

model = BasicGRU(1,256,vocab_size,1278,n_classes,0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

best_val_loss = None
for e in range(1,EPOCHS+1):
    train(model,optimizer,train_iter)
    val_loss, val_accuracy = evaluate(model,val_iter)

    print("[이폭: %d] 검증 오차: %5.2f | 검증 정확도: %5.2f" % (e,val_loss,val_accuracy))
    if not best_val_loss or val_loss < best_val_loss:
        print("모델 저장")
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(),"snapshot/txtclassification.pt")
        best_val_loss = val_loss

Building Basic GRU model...
[이폭: 1] 검증 오차:  0.59 | 검증 정확도: 71.32
모델 저장
[이폭: 2] 검증 오차:  0.36 | 검증 정확도: 84.62
모델 저장
[이폭: 3] 검증 오차:  0.39 | 검증 정확도: 84.82
[이폭: 4] 검증 오차:  0.47 | 검증 정확도: 85.92
[이폭: 5] 검증 오차:  0.67 | 검증 정확도: 83.26
[이폭: 6] 검증 오차:  0.58 | 검증 정확도: 84.70


## 4-3. Seq2Seq 모델 구현하기 (미니 seq2seq)
 . 간단한 구현을 위해 미니 seq2seq를 구현하므로 일반적인 단어단위 인베딩이 아닌 글자 단위 캐릭터 임베딩을 사용   
 . 아스키 코드 사용 => 사전 토큰수 256(아스키 코드의 수)

In [4]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt

vocab_size = 256
x_ = list(map(ord,"hello")) ## ord는 파이썬 내장 함수로 유니코드 코드 포인트를 나타내는 정수로 변환
y_ = list(map(ord,"hola"))
x = torch.LongTensor(x_)
y = torch.LongTensor(y_)

class Seq2Seq(nn.Module):
    def __init__(self,vocab_size,hidden_size):
        super(Seq2Seq,self).__init__()
        self.n_layers = 1
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,hidden_size)
        self.encoder = nn.GRU(hidden_size,hidden_size)        
        self.decoder = nn.GRU(hidden_size,hidden_size)
        self.project = nn.Linear(hidden_size,vocab_size)
    
    def forward(self,inputs,targets):
        initial_state = self._init_state()
        embedding = self.embedding(inputs).unsqueeze(1) ## 1차원 추가
        encoder_output, encoder_state = self.encoder(embedding,initial_state)
        ## 문맥벡터 encoder_state랑 decoder_input을 넣어주면서 디코더를 돌림
        ## 맨 처음 디코더 인풋은 문장 시작 토큰을 입력 데이터로 넣어야함 => 여기서는 0
        decoder_state = encoder_state
        decoder_input = torch.LongTensor([0])
        outputs = []
        ## 여기선 for문 사용
        ## 실제 번역기에선 for문을 사용하지 않고 <en> <fr> <es> 등의 토큰을 사용하여 최종 출력을 얻음
        for i in range(targets.size()[0]):
            decoder_input = self.embedding(decoder_input).unsqueeze(1)
            decoder_output, decoder_state = self.decoder(decoder_input,decoder_state)
            ## 디코더의 출력을 선형 레이어를 통과시켜서 다음 단어를 예측
            projection = self.project(decoder_output)
            outputs.append(projection)
            ## 티처 포싱 방법
            ## 디코더의 출력을 다음 입력으로 넣어주는 것이 아니라 실제 목표값을 넣어주는 방법
            ## 디코더의 출력을 넣어주어서 학습시키면 디코더의 출력값이 잘못된 값이 나올 수 도 있으므로
            ## 학습이 느려진다. 그러므로 학습 때는 실제 정답을 넣어줘서 학습을 빨리 시킨다
            ## 하지만 실제 예측할 때는 디코더의 출력값을 넣어줘야 한다!
            decoder_input = torch.LongTensor([targets[i]])
        outputs = torch.stack(outputs).squeeze()        
        return outputs

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

seq2seq = Seq2Seq(vocab_size,16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq2seq.parameters(),lr=0.001)

log = []
for i in range(1000):
    prediction = seq2seq(x,y)
    loss = criterion(prediction,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_val = loss.data
    log.append(loss_val)
    if i % 100 == 0:
        print("\n 반복:%d 오차:%s" % (i,loss_val.item()))
        _, top1 = prediction.data.topk(1,1)
        print([chr(c) for c in top1.squeeze().numpy().tolist()])



 반복:0 오차:5.697714805603027
['ñ', '¡', 'ô', 'Ü']

 반복:100 오차:2.1441774368286133
['h', 'h', 'l', 'a']

 반복:200 오차:0.7406474947929382
['h', 'h', 'l', 'a']

 반복:300 오차:0.3772960603237152
['h', 'o', 'l', 'a']

 반복:400 오차:0.24323907494544983
['h', 'o', 'l', 'a']

 반복:500 오차:0.17518796026706696
['h', 'o', 'l', 'a']

 반복:600 오차:0.13386446237564087
['h', 'o', 'l', 'a']

 반복:700 오차:0.1063300222158432
['h', 'o', 'l', 'a']

 반복:800 오차:0.08686168491840363
['h', 'o', 'l', 'a']

 반복:900 오차:0.07248760014772415
['h', 'o', 'l', 'a']
